In [ ]:
X = df[['MonthOfYear', 'DayOfYear', 'WeekOfYear',"TimeOfDay","Temp"]]
y = df.iloc[:,2:3]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
import xgboost
from xgboost import XGBRegressor
xgb = XGBRegressor().fit(X_train, y_train)
y_pred = xgb.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

xgb = XGBRegressor()
xgb_params = {"learning_rate": [0.1,0.01,0.5],
             "max_depth": [2,3,4,5,8],
             "n_estimators": [100,200,500,1000],
             "colsample_bytree": [0.4,0.7,1]
             }

xgb_cv_model  = GridSearchCV(xgb,xgb_params, cv = 5, verbose = 2).fit(X_train, y_train)

xgb_cv_model.best_params_

xgb_tuned = XGBRegressor(colsample_bytree=0.7,learning_rate=0.1,max_depth=5,n_estimators=1000).fit(X_train,y_train)

y_pred = xgb_tuned.predict(X_test)
print("mse = ",np.sqrt(mean_squared_error(y_test,y_pred)))
model=r2_score(y_test, y_pred)
print('r2 = {}'.format(model))

df.index=df.iloc[:,0]
df.index = pd.to_datetime(df.index)
# Predicting for whole dataset
df['y_pred'] =  xgb_tuned.predict(df[['MonthOfYear', 'DayOfYear', 'WeekOfYear',"TimeOfDay","Temp"]])

# Create Interactive Plot Using Bokeh
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Range1d
output_notebook() 

p = figure(plot_width=800, 
           plot_height=400,
           title="Evsel Elektrik Güç Tüketim Tahmini",
           x_axis_label='Tarih ve Zaman',
           y_axis_label='Elektrik Tüketimi[kW]',
           x_axis_type="datetime")

p.line(df.index, df['Load'], legend='Gözlemlenen Elektrik Tüketimi', line_width=1)
p.line(df.index, df['y_pred'], legend='Tahmin Edilen Elektrik Tüketimi', line_width=1, line_color="red")

show(p, notebook_handle = True)

# baggingli
Importance = pd.DataFrame({'Importance':xgb_tuned.feature_importances_*100}, 
                          index = X_train.columns)


Importance.sort_values(by = 'Importance', 
                       axis = 0, 
                       ascending = True).plot(kind = 'barh', 
                                              color = 'r', )

plt.xlabel('Variable Importance')
plt.gca().legend_ = None